# Dimension Modeling

In [0]:
%sql
select * from parquet.`abfss://silver@contosoprojectstorage.dfs.core.windows.net/contoso_sales`

/databricks/spark/python/pyspark/sql/dataframe.py:160: UserWarning: DataFrame constructor is internal. Do not directly use it.
  warnings.warn("DataFrame constructor is internal. Do not directly use it.")


com.databricks.backend.common.rpc.CommandCancelledException
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$5(SequenceExecutionState.scala:136)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3(SequenceExecutionState.scala:136)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3$adapted(SequenceExecutionState.scala:133)
	at scala.collection.immutable.Range.foreach(Range.scala:158)
	at com.databricks.spark.chauffeur.SequenceExecutionState.cancel(SequenceExecutionState.scala:133)
	at com.databricks.spark.chauffeur.ExecContextState.cancelRunningSequence(ExecContextState.scala:728)
	at com.databricks.spark.chauffeur.ExecContextState.$anonfun$cancel$1(ExecContextState.scala:446)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.ExecContextState.cancel(ExecContextState.scala:446)
	at com.databricks.spark.chauffeur.ExecutionContextManagerV1.can

## Dim_model Sink - Initial and Incremental run

In [0]:
from pyspark.sql import functions as F
from pyspark.sql.types import DateType, IntegerType, LongType
from pyspark.sql.window import Window

# Get source data
df_src = spark.sql('''
    SELECT DISTINCT             subcategory_key,
                                subcategory_name
    FROM parquet.`abfss://silver@contosoprojectstorage.dfs.core.windows.net/contoso_sales`
''')

# Initialize target DataFrame
if spark.catalog.tableExists('contoso_catalog.gold.dim_subcategories'):
    df_tgt = spark.sql('''
    SELECT                      surrogate_key, 
                                subcategory_key,
                                subcategory_name
        FROM contoso_catalog.gold.dim_subcategories
    ''')
else:
    df_tgt = df_src.withColumn("surrogate_key", F.lit(0).cast(LongType()))\
                   .withColumn("is_current", F.lit(1).cast(IntegerType()))\
                   .withColumn("start_date", F.current_date())\
                   .withColumn("end_date", F.lit(None).cast(DateType()))\
                   .filter("1 = 0")  # Empty Schema

def generate_surrogate_key(df, start_value):
    w = Window.orderBy("subcategory_key")
    return df.withColumn("surrogate_key", F.row_number().over(w) + F.lit(start_value))

def apply_scd_type2_changes(df_src, df_tgt):
    if df_tgt.rdd.isEmpty():
        return generate_surrogate_key(df_src, 0)\
               .withColumn("start_date", F.current_date())\
               .withColumn("end_date", F.lit(None).cast(DateType()))\
               .withColumn("is_current", F.lit(1).cast(IntegerType()))

    max_surrogate_key = df_tgt.agg(F.max("surrogate_key")).collect()[0][0]
    
    src = df_src.alias("src")
    tgt = df_tgt.filter(F.col("is_current") == 1).alias("tgt")
    
    joined_df = src.join(tgt, "subcategory_key", "outer")
    
    new_records = joined_df.filter(F.col("tgt.surrogate_key").isNull())
    
    if new_records.count() > 0:
        new_records = generate_surrogate_key(new_records, max_surrogate_key + 1)\
                     .withColumn("start_date", F.current_date())\
                     .withColumn("end_date", F.lit(None).cast(DateType()))\
                     .withColumn("is_current", F.lit(1).cast(IntegerType()))

    changed_records = joined_df.filter((F.col("tgt.surrogate_key").isNotNull()) &
                                       (
                                        (F.coalesce(src.subcategory_name != tgt.subcategory_name, F.lit(False)))|
                                        
                                       )
                                        )   
    
    if changed_records.count() > 0:
        new_versions = changed_records.select("src.subcategory_key", "src.subcategory_name")
        
        start_key = max_surrogate_key + (new_records.count() if new_records.count() > 0 else 0) + 1
        new_versions = generate_surrogate_key(new_versions, start_key)\
                       .withColumn("start_date", F.current_date())\
                       .withColumn("end_date", F.lit(None).cast(DateType()))\
                       .withColumn("is_current", F.lit(1).cast(IntegerType()))

        old_versions = df_tgt.join(changed_records.select("subcategory_key"), "subcategory_key", "inner")\
                             .withColumn("end_date", F.when(F.col("is_current") == 1, F.date_sub(F.current_date(), 1)).otherwise(F.col("end_date")))\
                             .withColumn("is_current", F.when(F.col("is_current") == 1, F.lit(0)).otherwise(F.col("is_current")))

        unchanged_records = df_tgt.join(changed_records.select("subcategory_key"), "subcategory_key", "leftanti")
        
        if new_records.count() > 0:
            final_df = unchanged_records\
                                    .unionAll(old_versions)\
                                    .unionAll(new_records)\
                                    .unionAll(new_versions)
        else:
            final_df = unchanged_records\
                                    .unionAll(old_versions)\
                                    .unionAll(new_versions)

    
    return final_df

# Apply SCD Type 2 changes
result_df = apply_scd_type2_changes(df_src, df_tgt)

display(result_df)



## Write data to catalog

In [0]:
final_df.write.format('delta')\
                .mode('overwrite')\
                    .option('path', 'abfss://gold@contosoprojectstorage.dfs.core.windows.net/dim_subcategories')\
                        .saveAsTable('contoso_catalog.gold.dim_subcategories')

In [0]:
%sql
select * from contoso_catalog.gold.dim_subcategories

subcategory_key,subcategory_name,is_current,start_date,end_date
101,MP4&MP3,1,2025-01-10,null
104,Recording Pen,1,2025-01-10,null
106,Bluetooth Headphones,1,2025-01-10,null
201,Televisions,1,2025-01-10,null
202,VCD & DVD,1,2025-01-10,null
203,Home Theater System,1,2025-01-10,null
205,Car Video,1,2025-01-10,null
301,Laptops,1,2025-01-10,null
303,Desktops,1,2025-01-10,null
304,Monitors,1,2025-01-10,null
